In [2]:
import pandas as pd

In [3]:
df = pd.read_csv(r"C:\Mini_H\kickstarter\data\kickstarter.csv")

In [3]:
print(df["state"].unique())

['successful' 'failed' 'live' 'canceled' 'submitted' 'started' 'suspended']


In [4]:
## karena hanya butuh binary saja sukses atau tidak jadi ambil hanya 2

df_model = df[df["state"].isin(["successful", "failed"])].copy()

In [5]:
## encode target
df_model["target"] = (df_model["state"] == "successful").astype(int)

print("Shape setelah filter:", df_model.shape)
print(df_model["target"].value_counts(normalize=True))

Shape setelah filter: (222350, 43)
target
1    0.656443
0    0.343557
Name: proportion, dtype: float64


In [6]:
## rubah timestampp ke datetime
df_model["launched_at"] = pd.to_datetime(df_model["launched_at"], unit="s")
df_model["deadline"] = pd.to_datetime(df_model["deadline"], unit="s")

In [7]:
## durasi kampanye
df_model["duration_days"] = (df_model["deadline"] - df_model["launched_at"]).dt.days

In [11]:
## target goal dalam USD
df_model["usd_goal_real"] = df_model["goal"] * df_model["static_usd_rate"]

In [12]:
## panjang tesk bulb
df_model["title_len"] = df_model["name"].astype(str).str.len()
df_model["blurb_len"] = df_model["blurb"].astype(str).str.len()

In [13]:
print(df_model[["duration_days","usd_goal_real","title_len","blurb_len"]].describe())

       duration_days  usd_goal_real      title_len      blurb_len
count  222350.000000   2.223500e+05  222350.000000  222350.000000
mean       32.924605   3.128449e+04      35.506530     103.807826
std        12.706842   9.282272e+05      15.545039      31.653932
min         1.000000   1.000000e-02       1.000000       1.000000
25%        29.000000   1.217983e+03      22.000000      83.000000
50%        30.000000   4.150059e+03      35.000000     116.000000
75%        35.000000   1.080544e+04      49.000000     130.000000
max       120.000000   1.314698e+08      85.000000     151.000000


1. duration_days (lama kampanye)  
Rata-rata: 32.9 hari → sesuai pola Kickstarter (kampanye biasanya 30 hari).  
Min: 1 hari → mungkin kampanye eksperimen / data error.  
Max: 120 hari → ada kampanye yang panjang sekali (padahal jarang sukses).  
Insight: Mayoritas kampanye berlangsung 29–35 hari, menunjukkan standar optimal sekitar sebulan.  
  
2. usd_goal_real (target dana USD)  
Median: $4,150 (cukup realistis).  
Mean: $31,284 → jauh lebih tinggi dari median, artinya ada outlier target sangat besar.  
Max: $131 juta 😅 → jelas outlier, biasanya proyek ambisius yang gagal.  
Insight: Target dana realistis cenderung di bawah $10k, sementara target ratusan ribu/juta jarang tercapai.  
  
3. title_len (panjang judul)  
Median: 35 karakter → sekitar 5–6 kata, cukup ringkas.  
Max: 85 karakter → judul panjang sekali, bisa membingungkan.  
Insight: Judul proyek idealnya 20–50 karakter (padat, jelas, mudah diingat).  
  
4. blurb_len (panjang deskripsi singkat)  
Median: 116 karakter → cukup detail untuk pitch singkat.  
Max: 151 karakter (mungkin ada batas dari Kickstarter).  
Insight: Blurb biasanya 80–130 karakter, cukup untuk menjelaskan inti proyek.  
  
Kesimpulan sementara :  
Mayoritas kampanye sesuai “aturan emas” Kickstarter: durasi ~30 hari, target goal moderat, judul ringkas, blurb jelas.  
Outlier ada pada target dana sangat tinggi (hingga >100 juta USD) → perlu di-handle supaya model tidak bias.  
Fitur yang kita buat sudah masuk akal untuk prediksi sukses/gagal.